## TABLE OF CONTENTS

1. Libraries
2. PG Admin Functions
	
    1. Triggers & Trigger Functions

3. PG Custom Geo-Functions
4. TABLES:
	
    1. NYC Tables
    2. Other Tables

5. Custom PostGIS Functions
6. Useful SQL/PG SQL Queries
7. Useful PostGIS Functions:

    1. Conversion Functions
	2. Geometry Constructors
	3. Geometry Editors
	4. Geometry Outputs
	5. Operators
	6. Relationships & Measurements
	7. Geometry Processing

# Libraries

In [ ]:
# IPython Options
# %matplotlib inline
# %qtconsole
%pdb on

In [ ]:
# Change Encoding
import sys
import codecs
reload(sys)
#sys.setdefaultencoding('UTF8')
sys.setdefaultencoding('ascii')

In [14]:
# Load Libraries
from sys import path as py_path
from f_postgres import pgSQL
PG = pgSQL()
T = PG.T

def to_sql(cmd):
    T.conn.set_isolation_level(0)
    T.cur.execute(cmd)

%reload_ext sql
%sql postgresql://$T.DB_USER:$T.DB_PW@$T.DB_HOST:$T.DB_PORT/$T.DB_NAME
%config SqlMagic.feedback=True
%config SqlMagic.autopandas=True
%config SqlMagic.short_errors=False

# PG Admin Functions

In [ ]:
# get all tables from a database

database = raw_input('database?\t')
qry="""
select * from information_schema.tables 
WHERE table_schema='public'
order by table_name
"""
%sql postgres@$database $qry

In [ ]:
# get table columns/info

table = raw_input('table?\t')

qry = """   select column_name, data_type, character_maximum_length 
            from INFORMATION_SCHEMA.COLUMNS 
            where table_name = '%s'"""%table
%sql postgres@$T.DB_NAME $qry

In [ ]:
# get trigger information
database =  raw_input('database?\t')
qry="""
select relname, tgname, tgtype, proname, prosrc,
tgconstrrelid, tgdeferrable, tginitdeferred, tgnargs,
tgattr, tgargs from (pg_trigger join pg_class on tgrelid=pg_class.oid)
join pg_proc on (tgfoid=pg_proc.oid)
"""
%sql postgres@$T.DB_NAME $qry

In [ ]:
# de-dupe table
t = {'table':'',
     'uid_col':'',
     'partition_col':'',
     'sort_col':''}
for k in t.keys():
    v = raw_input(k + '?\t')
    t.update({k:v})

conn.set_isolation_level(0)
cur.execute(""" 
    drop table if exists tmp1;
    create table tmp1 as
        WITH res AS (
            SELECT
                t.%(uid_col)s t_%(uid_col)s,
                ROW_NUMBER() OVER(PARTITION BY t.%(partition_col)s
                                  ORDER BY %(sort_col)s ASC) AS rk
            FROM %(table)s t
            )
        SELECT s.t_%(uid_col)s t_%(uid_col)s
            FROM res s
            WHERE s.rk = 1;
    
    drop table if exists tmp2;
    create table tmp2 as
        select *
        from %(table)s t1
        WHERE EXISTS (select 1 from tmp1 t2 where t2.t_%(uid_col)s = t1.%(uid_col)s);
    drop table %(table)s;
    
    create table %(table)s as select * from tmp2;
    alter table %(table)s add primary key (%(uid_col)s);
    
    alter table tmp2 rename to %(table)s;
    
    drop table if exists tmp1;
    drop table if exists tmp2;
    """%t)

In [ ]:
# Execute a DO Function
qry="""
do language plpythonu 
$BODY$


from os import environ
for k,v in environ.iteritems():
    print k,v
    plpy.log(k,v)

import sys
import os
plpy.log(os.system('env')) 
plpy.log(sys.executable) 


    
$BODY$;
"""
%sql postgres@$T.DB_NAME $qry

## Triggers & Trigger Functions

In [ ]:
# Auto Add Primary Key
c="""
DROP FUNCTION if exists z_auto_add_primary_key();

CREATE OR REPLACE FUNCTION z_auto_add_primary_key()
  RETURNS event_trigger AS
$BODY$
DECLARE
    has_index boolean;
    tbl text;
    _seq text;
BEGIN
    has_index = (select relhasindex from pg_class 
            where relnamespace=2200 
            and relkind='r' 
            order by oid desc limit 1);

    IF (
        pg_trigger_depth()=0 
        and has_index=False )
    THEN
        tbl = (select relname t from pg_class 
            where relnamespace=2200 
            and relkind='r' 
            order by oid desc limit 1);
        _seq = format('%I_uid_seq',tbl);
        execute format('alter table %I add column uid serial primary key',tbl);
        execute format('alter table %I alter column uid set default z_next_free(
                    ''%I'', 
                    ''uid'', 
                    ''%I'')',tbl,tbl,_seq);
    end if;
END;
$BODY$
  LANGUAGE plpgsql;


DROP EVENT TRIGGER if exists missing_primary_key_trigger;

CREATE EVENT TRIGGER missing_primary_key_trigger 
ON ddl command end 
WHEN TAG IN ('CREATE TABLE')
EXECUTE PROCEDURE z_auto_add_primary_key();
"""
conn.set_isolation_level(0)
cur.execute(c)

In [ ]:
# Auto Add Last Updated
c="""
DROP FUNCTION if exists z_auto_add_last_updated_field() cascade;

CREATE OR REPLACE FUNCTION z_auto_add_last_updated_field()
  RETURNS event_trigger AS
$BODY$
DECLARE
    last_table text;
    has_last_updated boolean;
    
    
BEGIN
    last_table := (select relname from pg_class 
                 where relnamespace=2200 
                 and relkind='r' 
                 order by oid desc limit 1);
    
    SELECT count(*)>0 INTO has_last_updated FROM information_schema.columns
        where table_name='||quote_ident(last_table)||'
        and column_name='last_updated';

    IF (
        pg_trigger_depth()=0 
        and has_last_updated=False )
    THEN
        execute format('alter table %I add column last_updated timestamp with time zone',last_table);
    end if; 

END;
$BODY$
  LANGUAGE plpgsql;
"""
conn.set_isolation_level(0)
cur.execute(c)

c="""
DROP EVENT TRIGGER if exists missing_last_updated_field;

CREATE EVENT TRIGGER missing_last_updated_field 
ON ddl_command_end
WHEN TAG IN ('CREATE TABLE')
EXECUTE PROCEDURE z_auto_add_last_updated_field();
"""
conn.set_isolation_level(0)
cur.execute(c)

# PG Custom Geo-Functions

In [ ]:
# f(x) z_get_nodes_in_geom
%%sql postgres@routing
drop function if exists z_get_nodes_in_geom(text,text,geometry);
create or replace function z_get_nodes_in_geom(node_table text,
                                                node_col text,
                                                in_geom geometry)
returns text
as $$

    p = "SELECT st_collect(array_agg(%s)) geom FROM %s"%(node_col,node_table)
    multi_pts = plpy.execute(p)[0]['geom']

    p = plpy.prepare("SELECT st_collect(st_intersection($1,$2)) pts_within", ["geometry","geometry"]) 
    pts_within = plpy.execute(p,[ multi_pts,in_geom ])[0]['pts_within']

    _p = "SELECT id FROM %s WHERE %s"%(node_table,node_col)
    p = plpy.prepare(_p+" && $1", ["geometry"])
    return plpy.execute(p,[ pts_within ])[:]

$$ language plpythonu;

In [ ]:
# call z_get_nodes_in_geom
%%sql postgres@routing
select z_get_nodes_in_geom('lion_ways_slim_vertices_pgr',
                           'the_geom',
                           z_get_way_box('14 st', '10 ave', '59 st', '1 ave'))

In [ ]:
# f(x) z_update_table_with_closest_node
%%sql postgres@routing

drop function if exists public.z_update_table_with_closest_node(text,text,
                                              text,text,
                                              text,text);

create or replace function z_update_table_with_closest_node(node_table text, node_col text,
                                          update_table text, point_col text, 
                                          update_col text, update_col_idx text)
returns text
as $$
    
    p = "SELECT st_collect(array_agg(%s)) geom FROM %s"%(node_col,node_table)
    multi_pts = plpy.execute(p)[0]['geom']
    
    p = "select st_geomfromtext(%s,4326) geom,* from %s"%(point_col,update_table)
    for row in plpy.cursor(p):
        
        p = plpy.prepare("SELECT st_closestpoint($1,$2) close_pt", ["geometry","geometry"])
        close_pt = plpy.execute(p,[ multi_pts,row['geom'] ])[0]['close_pt']
        
        _p = "SELECT id FROM %s"%node_table
        p = plpy.prepare(_p+" WHERE the_geom = $1", ["geometry"])
        lion_node = plpy.execute(p,[ close_pt ])[0]['id']

        row_idx=row['%s'%update_col_idx]
        p = "UPDATE %s SET %s = %s where %s = %s"%(update_table,update_col,lion_node,update_col_idx,row_idx)
        z = plpy.execute(p)
        
    return 'ok'
        
$$ language plpythonu;

In [ ]:
# call z_update_table_with_closest_node
%%sql postgres@routing
select z_update_table_with_closest_node('lion_ways_slim_vertices_pgr','the_geom',
                                          'vendors', 'geom_text',
                                          'node', 'id')

In [ ]:
# f(x) z_get_closest_node_to_point_text
%%sql postgres@routing
drop function if exists public.z_get_closest_node_to_point_text(text,text,text);
create or replace function z_get_closest_node_to_point_text(node_table text,
                                                            node_col text,
                                                            uid_col text,    
                                                            to_point text)
returns integer
as $$
    
    p = "SELECT st_collect(array_agg(%s)) geom FROM %s"%(node_col,node_table)
    multi_pts = plpy.execute(p)[0]['geom']
    
    p = plpy.prepare("SELECT st_closestpoint($1,st_geomfromtext($2,4326)) close_pt", ["geometry","text"]) 
    close_pt = plpy.execute(p,[ multi_pts,to_point ])[0]['close_pt']
    
    _p = "SELECT id FROM %s"%node_table
    p = plpy.prepare(_p+" WHERE the_geom = $1", ["geometry"])
    return plpy.execute(p,[ close_pt ])[0]['id']
        
$$ language plpythonu;

In [ ]:
# call z_get_closest_node_to_point_text
%%sql postgres@routing 
select z_get_closest_node_to_point_text('lion_ways_slim_vertices_pgr','the_geom',
                      'POINT (-73.9893011318052771 40.7579070491531965)')

In [ ]:
### quick dijkstra
cmd = """SELECT cost FROM pgr_dijkstra('
                SELECT gid AS id,
                         source::integer,
                         target::integer,
                         len_miles::double precision AS cost
                        FROM lws',
                893, 4487, false, false);""".replace('\n','')
print pd.read_sql_query(cmd,engine)
# print round((pd.read_sql_query(cmd,engine).cost.sum()*3.280839895) / 5280,5),'miles between points'

In [ ]:
# create node_many_t
%%sql postgres@routing
DROP TYPE if exists node_many_t;
CREATE TYPE node_many_t AS (
    seq integer,
    source integer,
    target integer,
    cost double precision
);

In [ ]:
# f(x) z_get_miles_between_nodes(text,int,int)
%%sql postgres@routing
drop function if exists public.z_get_miles_between_nodes(text,integer,integer);
create or replace function z_get_miles_between_nodes(route_tbl text,
                                                     node_a integer,
                                                     node_b integer)
returns double precision
as $$
    
    p = """select sum(cost::double precision) arr
            FROM 
                (SELECT cost FROM pgr_dijkstra('
                    SELECT gid AS id,
                             source::integer,
                             target::integer,
                             len_miles::double precision AS cost
                            FROM %s',
                    %s, %s, false, false)) as all_costs"""%(route_tbl,node_a,node_b)
    
    return plpy.execute(p)[0]['arr']
        
$$ language plpythonu;

In [ ]:
# f(x) z_get_miles_between_nodes(text,integer,integer[])
%%sql postgres@routing
drop function if exists public.z_get_miles_between_nodes(text,
                                                        integer,
                                                        integer[]);
create or replace function z_get_miles_between_nodes(route_tbl text,
                                                     node_1 integer,
                                                     node_many integer[])
returns double precision[]
as $$
    
    p = """SELECT seq, id1 as source, id2 as target, cost::double precision
                FROM pgr_kdijkstraCost('
                SELECT gid AS id,
                         source::integer,
                         target::integer,
                         len_miles::double precision AS cost
                        FROM %s',
                %s, array%s, false, false);"""%(route_tbl,node_1,
                                                str([int(it) for it in node_many]))
    
    records = plpy.execute(p)
    return [ (r["cost"]) for r in records ]
        
$$ language plpythonu;

In [ ]:
# f(x) z_get_miles_between_nodes(text,integer[],integer[])
%%sql postgres@routing
drop function if exists public.z_get_miles_between_nodes(text,integer[],integer[]);
create or replace function z_get_miles_between_nodes(route_tbl text,
                                                     node_arr_1 integer[],
                                                     node_arr_2 integer[])
returns float[]
as $$
    r = []
    for i in range(0,len(node_arr_1)):
        p = """ SELECT sum(cost::double precision) arr
                FROM 
                    (SELECT cost FROM pgr_dijkstra('
                        SELECT gid AS id,
                                 source::integer,
                                 target::integer,
                                 len_miles::double precision AS cost
                                FROM %s',
                        %s, %s, false, false)) as all_costs"""%(route_tbl,
                                                                node_arr_1[i],
                                                                node_arr_2[i])
    
        r.append(plpy.execute(p)[0]['arr'])
    return r
        
$$ language plpythonu;

In [ ]:
# call z_get_miles_between_nodes
%%sql postgres@routing
select z_get_miles_between_nodes('lws',1722,8037)

In [ ]:
# f(x) z_update_table_with_closest_node
%%sql postgres@routing

drop function if exists public.z_add_point( text,
                                            double precision,
                                            double precision);

create or replace function z_add_point(pt_table text,
                                       pt_x double precision,
                                       pt_y double precision)
returns text
as $$
    
    p = "SELECT st_collect(array_agg(%s)) geom FROM %s"%(node_col,node_table)
    multi_pts = plpy.execute(p)[0]['geom']
    
    p = "select st_geomfromtext(%s,4326) geom,* from %s"%(point_col,update_table)
    for row in plpy.cursor(p):
        
        p = plpy.prepare("SELECT st_closestpoint($1,$2) close_pt", ["geometry","geometry"])
        close_pt = plpy.execute(p,[ multi_pts,row['geom'] ])[0]['close_pt']
        
        _p = "SELECT id FROM %s"%node_table
        p = plpy.prepare(_p+" WHERE the_geom = $1", ["geometry"])
        lion_node = plpy.execute(p,[ close_pt ])[0]['id']

        row_idx=row['%s'%update_col_idx]
        p = "UPDATE %s SET %s = %s where %s = %s"%(update_table,update_col,lion_node,update_col_idx,row_idx)
        z = plpy.execute(p)
        
    return 'ok'
        
$$ language plpythonu;

In [ ]:
# sql z_get_miles_bt_node_many
%%sql postgres@routing
drop function if exists public.z_get_miles_bt_node_many(integer,bigint[]);
create or replace function z_get_miles_bt_node_many(node_1 integer,
                                                    node_many bigint[])
returns table(seq integer,id1 integer,id2 integer,cost double precision)
as $$

SELECT p.seq, p.id1, p.id2, p.cost
    FROM pgr_kdijkstraCost(
        SELECT gid AS id,
        source::integer,
        target::integer,
        len_miles::double precision AS cost
        FROM lion_ways_slim',
    $1,
    array[1836, 1788, 1534, 2480, 2646, 1727, 128, 157, 1691, 1618],
    false,
    false) p;

$$ language sql;

# TABLES:

## NYC Tables

In [ ]:
# NYC TABLES -- Libraries & F(x)s 
from os                             import environ          as os_environ
from os                             import path             as os_path
from sys                            import path             as py_path
py_path.append(os_path.join(os_environ['BD'],'geolocation'))
from f_nyc_data import NYC_Tables
NT = NYC_Tables()

# NT.add_geom_using_address('seamless') # 'seamless' | 'yelp' | 'seamless_geom_error' | 'mnv'
# NT.add_geom_using_geocoding('seamless') # 'seamless' | 'yelp' | 'mnv'



In [ ]:
### match mnv to lot_pts (5697 rows affected, 50% successful)
conn.set_isolation_level(0)
cur.execute("""
update mnv v
set 
    bbl = l.bbl::integer,
    geom = l.geom
from lot_pts l
where l.bldg_street ilike v.clean_street
and (l.bldg_num_start <= v.building::integer and
     v.building::integer <= l.bldg_num_end)
and l.bbl is not null
and length( regexp_replace(v.building, '\d*', '') ) = 0
""")

In [ ]:
### add 'assess_com_val' to pluto *changed to 'assesstot_div_comarea'
p['assess_com_val'] = p.assesstot.map(float) / p.comarea.map(float)
engine.execute('drop table if exists tmp;')
p.to_sql('tmp',engine)
engine.execute("""

alter table pluto add column assess_com_val double precision;
update pluto set assess_com_val = t.assess_com_val
from tmp t
where gid = t.gid;
drop table if exists tmp;

""")

In [ ]:
### iter block for add_points_to_remaining_lots
show_some_detail=False
show_steps=False
engine.execute('drop table if exists temp;')
for block in chain(uniq_blocks[20:]):
#     res='\nblock\t',block
#     res+= " ".join(['  with',len(all_pts[all_pts.block==block]),'lots in lot_pts'])
#     res+= " ".join([print '  and',pd.read_sql("""
#                             select count(bbl) cnt
#                             from pluto
#                             where block = to_number('%s','99999')
#                         """%block,engine).cnt[0],'lots in pluto'])
    
    lot_pts,A,skip = [],[],False
    
    if show_some_detail==True: print 'block\t',block

    uniq_street = all_pts[all_pts.block==block].bldg_street.unique().tolist()
    if uniq_street.count('')!=0:    dev = uniq_street.pop(uniq_street.index(''))
    if uniq_street.count(None)!=0:  dev = uniq_street.pop(uniq_street.index(None))
        
    for street_name in chain(uniq_street):
        if show_some_detail==True: print '\t\tstreet_name\t',street_name

        lots = gd.read_postgis("""
                SELECT * FROM pluto 
                WHERE substring(to_char(bbl,'9999999999') from 3 for 5) = '%(block)s'
                AND street like concat(%(wildcard)s,'%(street)s')
                """%{'block':block,
                     'street':street_name,
                     'wildcard':'%s'},engine, params=['%'])

        if len(lots)!=0:

            #if show_steps==True: A.extend(lots.geom)
#             res+= " ".join([ '\t-',street_name,'with',len(lots),'lots'])

            ### Create Buffer Around Lots
            s = str([it.to_wkt() for it in lots.geom])
            T = {'buffer': '0.0005',
                 'geoms' : s}
            cmd =   """ SELECT ST_Buffer(ST_ConvexHull((ST_Collect(the_geom))), %(buffer)s) as geom
                        FROM ( SELECT (ST_Dump( unnest(array[%(geoms)s]) )).geom the_geom) as t""".replace('\n','')%T
            block_buffer = gd.read_postgis(cmd,engine)
            #if show_steps==True: A.extend(block_buffer.geom)

            ### Get street as Line from lion_ways
            cmd =   """ SELECT st_makeline(_geom) geom
                        FROM
                            st_geomfromtext(
                            ' %(block_buffer)s '
                            , 4326) block_buffer,
                            (select ( st_dump( geom )).geom _geom
                                FROM lion_ways l
                                WHERE l.clean_street = '%(1)s'
                                AND l.geom is not null  ) as t2
                        WHERE st_intersects(_geom,block_buffer) is True
                    """.replace('\n','')%{'1':str(street_name),
                                          'block_buffer':str(block_buffer.geom[0].to_wkt())}
            try:
                line_geom = gd.read_postgis(cmd,engine)
                skip_street=False
            except AttributeError:
                print 'skipping street:',street_name
                skip_street=True            

            if skip_street==False:

                # A.extend(line_geom.geom)
                LINE = line_geom.geom[0].to_wkt()

                ### Keep only part of line intersecting with buffer
                s1 = str([it.to_wkt() for it in block_buffer.geom][0])
                s2 = str([it.to_wkt() for it in line_geom.geom][0])
                T = {'block_buffer': s1,
                     'street_line' : s2}
                cmd =   """ SELECT res_geom geom
                            FROM
                                st_geomfromtext('%(block_buffer)s') s1,
                                st_geomfromtext('%(street_line)s') s2,
                                st_intersection(s1,s2) res_geom
                        """.replace('\n','')%T
                tmp_part_line=gd.read_postgis(cmd,engine)
        #
                if show_steps==True:  A.extend(tmp_part_line.geom)

                for i in range(0,len(lots.geom)):
                    bbl,lot = lots.ix[i,['bbl','geom']].values

                    cmd =   """ SELECT ( st_dump( _geom )).geom,ST_NPoints((( st_dump( _geom )).geom))
                            FROM (
                                SELECT ( st_dump( st_geomfromtext('%(1)s') )).geom _geom
                                ) as t
                        """.replace('\n','')%{'1':str(lot.to_wkt())}

                    ### Get Single Lot Polygon
                    lot=gd.read_postgis(cmd,engine)
                    if show_some_detail==True: A.append(lot.geom[0])

                    ### Get Line From Lot Polygon
                    perim_line = lot.boundary[0].to_wkt()
                    # if show_steps==True: A.append(lot.boundary[0])

                    ### Get Points of Segment of Lot Polygon Closest to Street
                    cmd =   """ SELECT ( st_dumppoints( st_geomfromtext('%(1)s') )).geom""".replace('\n','')%{'1':str(perim_line)}
                    points = gd.read_postgis(cmd,engine).geom
                    t1,t2={},{}
                    for j in range(1,len(points)):
                        poly_seg_pts=points[j-1],points[j]
                        dist_from_street = pd.read_sql("""  select
                                                            st_distance(
                                                               st_geomfromtext(' %(start_pt)s '),
                                                               st_geomfromtext(' %(street)s '))
                                                               +
                                                            st_distance(
                                                               st_geomfromtext(' %(end_pt)s '),
                                                               st_geomfromtext(' %(street)s '))
                                                            dist
                                                        """.replace('\n','')%{'start_pt':str(poly_seg_pts[0]),
                                                                              'end_pt' : str(poly_seg_pts[1]),
                                                                              'street' : LINE},engine)
                        t1.update({j:dist_from_street.dist[0]})
                        t2.update({j:poly_seg_pts})
                    closest_seg_pts = t2[t1.values().index(min(t1.values()))+1]

                    ### Lot Segment MidPoint
                    lot_seg_mid_pt = gd.read_postgis("""   SELECT ST_Line_Interpolate_Point(st_makeline(ptA,ptB),0.5) geom
                                                            FROM
                                                                st_geomfromtext(' %(start_pt)s ') ptA,
                                                                st_geomfromtext(' %(end_pt)s ') ptB
                                                    """.replace('\n','')%{'start_pt': str(closest_seg_pts[0]),
                                                                          'end_pt'  : str(closest_seg_pts[1]),
                                                                          'street'  : LINE},engine)

                    if show_steps==True: A.append(lot_seg_mid_pt.geom[0])

                    ### Closest Point in Street
                    street_seg_mid_pt = gd.read_postgis("""
                        SELECT ST_ClosestPoint(
                            st_geomfromtext(' %(street)s '),
                            st_geomfromtext(' %(mid_pt)s ')) geom
                        """.replace('\n','')%{'mid_pt'  : str(lot_seg_mid_pt.geom[0].to_wkt()),
                                              'street'  : LINE},engine)
                    #A.append(street_seg_mid_pt.geom[0])

                    ### Absolute Angle of Segment (12=0 deg.,6=180 deg.)
                    seg_angle = pd.read_sql("""     SELECT ST_Azimuth(ptA,ptB) ang
                                                    FROM
                                                                st_geomfromtext(' %(start_pt)s ') ptA,
                                                                st_geomfromtext(' %(end_pt)s ') ptB
                                                    """.replace('\n','')%{'start_pt': str(closest_seg_pts[0]),
                                                                          'end_pt'  : str(closest_seg_pts[1])},engine).ang[0]

                    ### Point on Street that Intersects with perp. line extending from Poly Segment Midpoint
                    this_lot_pt = gd.read_postgis("""
                        SELECT ( st_dumppoints( st_intersection( street, st_makeline(
                            st_makeline(mid_pt::geometry(Point,4326),ptA::geometry(Point,4326)),
                            st_makeline(mid_pt::geometry(Point,4326),ptB::geometry(Point,4326))))  )).geom
                        FROM
                            st_geomfromtext(' %(mid_pt)s ', 4326) mid_pt,
                            st_geomfromtext(' %(street)s ', 4326) street,
                            st_geomfromtext(' %(line_seg_pt)s ', 4326) line_seg_pt,
                            ST_Distance_Spheroid(mid_pt,line_seg_pt,
                                'SPHEROID["WGS 84",6378137,298.257223563]') dist,
                            ST_Project(mid_pt,dist+(dist*1.1),%(ang1)s) ptA,
                            ST_Project(mid_pt,dist+(dist*1.1),%(ang2)s) ptB
                                                    """.replace('\n','')%{'mid_pt' : str(lot_seg_mid_pt.geom[0].to_wkt()),
                                                                          'line_seg_pt' : str(street_seg_mid_pt.geom[0].to_wkt()),
                                                                          'street'  : LINE,
                                                                          'ang1' : str(deg_to_rad(90-(360-rad_to_deg(seg_angle)))),
                                                                          'ang2' : str(deg_to_rad(270-(360-rad_to_deg(seg_angle))))},
                                                  engine).geom
                #
                    if len(this_lot_pt)==0 or len(lot.geom)>1:
                        this_lot_pt = gd.read_postgis("""select st_closestpoint(
                                                st_geomfromtext(' %(street)s '),
                                                st_geomfromtext(' %(lot)s ')) geom
                            """%{'street'  : LINE,
                                 'lot' : str(lot.geom[0].to_wkt())},engine).geom

                        if show_some_detail==True: A.append(this_lot_pt[0]) # taking first value...
                        lot_pts.append({'bbl':bbl,
                                        'geom':this_lot_pt[0]})

                    else:

                        if show_some_detail==True: A.append(this_lot_pt[0]) # taking first value...
                        lot_pts.append({'bbl':bbl,
                                        'geom':this_lot_pt[0]})

                        ### Line Connecting Lot to Street
                        line_lot_to_street = gd.read_postgis("""
                            SELECT ST_ShortestLine(lot_pt,mid_pt) geom
                            FROM
                                st_geomfromtext(' %(lot_pt)s ',4326) lot_pt,
                                st_geomfromtext(' %(mid_pt)s ',4326) mid_pt
                            """.replace('\n','')%{'mid_pt'  : str(lot_seg_mid_pt.geom[0].to_wkt()),
                                                  'lot_pt'  : this_lot_pt[0]},engine)
                #
                        if len(line_lot_to_street.geom)>1:
                            gd.GeoSeries(line_lot_to_street.geom).plot()
                            print 'too much for Line Connecting Lot to Street'
                            raise SystemError
                        if show_some_detail==True: A.append(line_lot_to_street.geom[0])

    if show_some_detail==True: 
        d=gd.GeoSeries(A).plot(fig_size=(26,22),
                               save_fig_path=save_fig_path,
                               save_and_show=False)
        me = raw_input('y?')
    else: me='y'

    if (me=='y' or me=='yq') and lot_pts!=[]:
        c = gd.GeoDataFrame(lot_pts)
        d=[]
        dev=[d.extend(zip(*it.coords.xy)) for it in c.geom]
        D = pd.DataFrame({'bbl':c.bbl.tolist(),'xy':d})
        D['x'],D['y'] = zip(*D.xy)
        D.to_sql('temp',engine,if_exists='append')

#         print '  + added',len(lot_pts),'lots'
        if me=='yq': raise SystemError
        else:
            lot_pts,A,skip = [],[],False
            plt.clf()
    else:
        if me!='y': raise SystemExit


engine.execute("""
UPDATE lot_pts l
SET geom = st_setsrid(st_makepoint(t.x,t.y),4326) 
FROM temp t 
WHERE l.bbl = t.bbl
""")
print 'DONE!'

In [ ]:
### add_points_to_remaining_lots
show_some_detail=False
show_steps=False
from IPython.display import FileLink, FileLinks
save_fig_path = '/Users/admin/Projects/GIS/matplotlib_data/nyc_block_and_lot.png'
matplot_files = FileLinks("/Users/admin/Projects/GIS/matplotlib_data/")
# display(matplot_files)
from math import degrees as rad_to_deg
from math import radians as deg_to_rad
from itertools import chain
from matplotlib import pyplot as plt

def geoms_to_collection(geoms):
    s='GEOMETRYCOLLECTION('
    for it in geoms:
        s+=it.to_wkt()+','
    return s.rstrip(',')+')'
def geoms_to_text(geoms):
    if type(geoms_to_text) != list: geoms=list(geoms)
    s=''
    for it in geoms:
        try:
            s+="ST_GeomFromText('"+it.to_wkt()+"'),"
        except:
            s+="ST_GeomFromText('"+it+"'),"
    return s.rstrip(',')
def geom_txts_to_collection(geom_txts):
    return "ST_Collect(ARRAY["+geoms_to_text(geom_txts)+'])'

all_pts = pd.read_sql_query("select * from lot_pts where geom is null and ignore is false and place is false",engine)
all_pts['block'] = all_pts.bbl.map(lambda s: str(s)[1:6])
uniq_blocks = all_pts.block.unique().tolist()
if show_some_detail==True: print len(uniq_blocks),'unique blocks'
if show_some_detail==True: print 'pluto has 1961 unique blocks'
# a=pd.read_sql_query("select distinct block from pluto",engine).block.tolist()
# for it in a:
#     if uniq_blocks.count(str('%05d'%it))==0:
#         print it
if show_some_detail==True: print 'lot 656 is a pier and was removed from lot_pts'

#

In [ ]:
### Make "address_idx" and Update "lot_pts" with ranges and lot_idx

from f_postgres import make_index,add_bldg_range_pts
from f_postgres import create_lot_idx_start_pts,create_lot_idx_end_pts

make_index(describe=False,commit=True)
add_bldg_range_pts()
create_lot_idx_start_pts()
create_lot_idx_end_pts()

In [ ]:
### add_points_to_remaining_lots
show_some_detail=False
show_steps=False
from IPython.display import FileLink, FileLinks
save_fig_path = '/Users/admin/Projects/GIS/matplotlib_data/nyc_block_and_lot.png'
matplot_files = FileLinks("/Users/admin/Projects/GIS/matplotlib_data/")
# display(matplot_files)
from math import degrees as rad_to_deg
from math import radians as deg_to_rad
from itertools import chain
from matplotlib import pyplot as plt

def geoms_to_collection(geoms):
    s='GEOMETRYCOLLECTION('
    for it in geoms:
        s+=it.to_wkt()+','
    return s.rstrip(',')+')'
def geoms_to_text(geoms):
    if type(geoms_to_text) != list: geoms=list(geoms)
    s=''
    for it in geoms:
        try:
            s+="ST_GeomFromText('"+it.to_wkt()+"'),"
        except:
            s+="ST_GeomFromText('"+it+"'),"
    return s.rstrip(',')
def geom_txts_to_collection(geom_txts):
    return "ST_Collect(ARRAY["+geoms_to_text(geom_txts)+'])'

all_pts = pd.read_sql_query("select * from lot_pts where geom is null and ignore is false and place is false",engine)
all_pts['block'] = all_pts.bbl.map(lambda s: str(s)[1:6])
uniq_blocks = all_pts.block.unique().tolist()
if show_some_detail==True: print len(uniq_blocks),'unique blocks'
if show_some_detail==True: print 'pluto has 1961 unique blocks'
# a=pd.read_sql_query("select distinct block from pluto",engine).block.tolist()
# for it in a:
#     if uniq_blocks.count(str('%05d'%it))==0:
#         print it
if show_some_detail==True: print 'lot 656 is a pier and was removed from lot_pts'

#
### iter block for add_points_to_remaining_lots
engine.execute('drop table if exists temp;')
for block in chain(uniq_blocks[:5]):
    print 'block\t',block
    
    lot_pts,A,skip = [],[],False
    if show_some_detail==True: print 'block\t',block

    uniq_street = all_pts[all_pts.block==block].bldg_street.unique().tolist()
    if uniq_street.count('')!=0:    dev = uniq_street.pop(uniq_street.index(''))
    if uniq_street.count(None)!=0:  dev = uniq_street.pop(uniq_street.index(None))

    for street_name in chain(uniq_street):
        if show_some_detail==True: print 'street_name\t',street_name

        ### Get All Lots on Block and On This Street
#         pts = all_pts[(all_pts.block==block)&(all_pts.bldg_street==street_name)].sort('bldg_num').reset_index(drop=True)
#         cmd=""" SELECT _bbl bbl,geom
#                 FROM pluto p,unnest(array%s) _bbl
#                 WHERE p.bbl = _bbl"""%str(pts.bbl.astype(int).tolist()).replace("'",'')
        lots = gd.read_postgis("""
                select * from pluto 
                where substring(to_char(bbl,'9999999999') from 3 for 5) = '%s'
                """%block,engine)

#         lots = gd.read_postgis(cmd,engine)
        #if show_steps==True: A.extend(lots.geom)
        print '\t\t',len(lots),'lots'

        ### Create Buffer Around Lots
        s = str([it.to_wkt() for it in lots.geom])
        T = {'buffer': '0.0005',
             'geoms' : s}
        cmd =   """ SELECT ST_Buffer(ST_ConvexHull((ST_Collect(the_geom))), %(buffer)s) as geom
                    FROM ( SELECT (ST_Dump( unnest(array[%(geoms)s]) )).geom the_geom) as t""".replace('\n','')%T
        block_buffer = gd.read_postgis(cmd,engine)
        #if show_steps==True: A.extend(block_buffer.geom)

        ### Get street as Line from lion_ways
        cmd =   """ SELECT st_makeline(_geom) geom
                    FROM
                        st_geomfromtext(
                        ' %(block_buffer)s '
                        , 4326) block_buffer,
                        (select ( st_dump( geom )).geom _geom
                            FROM lion_ways l
                            WHERE l.clean_street = '%(1)s'
                            AND l.geom is not null  ) as t2
                    WHERE st_intersects(_geom,block_buffer) is True
                """.replace('\n','')%{'1':str(street_name),
                                      'block_buffer':str(block_buffer.geom[0].to_wkt())}
        try:
            line_geom = gd.read_postgis(cmd,engine)
            skip_street=False
        except AttributeError:
            print 'skipping street:',street_name
            skip_street=True

        if skip_street==False:

            # A.extend(line_geom.geom)
            LINE = line_geom.geom[0].to_wkt()

            ### Keep only part of line intersecting with buffer
            s1 = str([it.to_wkt() for it in block_buffer.geom][0])
            s2 = str([it.to_wkt() for it in line_geom.geom][0])
            T = {'block_buffer': s1,
                 'street_line' : s2}
            cmd =   """ SELECT res_geom geom
                        FROM
                            st_geomfromtext('%(block_buffer)s') s1,
                            st_geomfromtext('%(street_line)s') s2,
                            st_intersection(s1,s2) res_geom
                    """.replace('\n','')%T
            tmp_part_line=gd.read_postgis(cmd,engine)
    #
            if show_steps==True:  A.extend(tmp_part_line.geom)

            for i in range(0,len(lots.geom)):
                bbl,lot = lots.ix[i,['bbl','geom']].values

                cmd =   """ SELECT ( st_dump( _geom )).geom,ST_NPoints((( st_dump( _geom )).geom))
                        FROM (
                            SELECT ( st_dump( st_geomfromtext('%(1)s') )).geom _geom
                            ) as t
                    """.replace('\n','')%{'1':str(lot.to_wkt())}

                ### Get Single Lot Polygon
                lot=gd.read_postgis(cmd,engine)
                if show_some_detail==True: A.append(lot.geom[0])

                ### Get Line From Lot Polygon
                perim_line = lot.boundary[0].to_wkt()
                # if show_steps==True: A.append(lot.boundary[0])

                ### Get Points of Segment of Lot Polygon Closest to Street
                cmd =   """ SELECT ( st_dumppoints( st_geomfromtext('%(1)s') )).geom""".replace('\n','')%{'1':str(perim_line)}
                points = gd.read_postgis(cmd,engine).geom
                t1,t2={},{}
                for j in range(1,len(points)):
                    poly_seg_pts=points[j-1],points[j]
                    dist_from_street = pd.read_sql("""  select
                                                        st_distance(
                                                           st_geomfromtext(' %(start_pt)s '),
                                                           st_geomfromtext(' %(street)s '))
                                                           +
                                                        st_distance(
                                                           st_geomfromtext(' %(end_pt)s '),
                                                           st_geomfromtext(' %(street)s '))
                                                        dist
                                                    """.replace('\n','')%{'start_pt':str(poly_seg_pts[0]),
                                                                          'end_pt' : str(poly_seg_pts[1]),
                                                                          'street' : LINE},engine)
                    t1.update({j:dist_from_street.dist[0]})
                    t2.update({j:poly_seg_pts})
                closest_seg_pts = t2[t1.values().index(min(t1.values()))+1]

                ### Lot Segment MidPoint
                lot_seg_mid_pt = gd.read_postgis("""   SELECT ST_Line_Interpolate_Point(st_makeline(ptA,ptB),0.5) geom
                                                        FROM
                                                            st_geomfromtext(' %(start_pt)s ') ptA,
                                                            st_geomfromtext(' %(end_pt)s ') ptB
                                                """.replace('\n','')%{'start_pt': str(closest_seg_pts[0]),
                                                                      'end_pt'  : str(closest_seg_pts[1]),
                                                                      'street'  : LINE},engine)

                if show_steps==True: A.append(lot_seg_mid_pt.geom[0])

                ### Closest Point in Street
                street_seg_mid_pt = gd.read_postgis("""
                    SELECT ST_ClosestPoint(
                        st_geomfromtext(' %(street)s '),
                        st_geomfromtext(' %(mid_pt)s ')) geom
                    """.replace('\n','')%{'mid_pt'  : str(lot_seg_mid_pt.geom[0].to_wkt()),
                                          'street'  : LINE},engine)
                #A.append(street_seg_mid_pt.geom[0])

                ### Absolute Angle of Segment (12=0 deg.,6=180 deg.)
                seg_angle = pd.read_sql("""     SELECT ST_Azimuth(ptA,ptB) ang
                                                FROM
                                                            st_geomfromtext(' %(start_pt)s ') ptA,
                                                            st_geomfromtext(' %(end_pt)s ') ptB
                                                """.replace('\n','')%{'start_pt': str(closest_seg_pts[0]),
                                                                      'end_pt'  : str(closest_seg_pts[1])},engine).ang[0]

                ### Point on Street that Intersects with perp. line extending from Poly Segment Midpoint
                this_lot_pt = gd.read_postgis("""
                    SELECT ( st_dumppoints( st_intersection( street, st_makeline(
                        st_makeline(mid_pt::geometry(Point,4326),ptA::geometry(Point,4326)),
                        st_makeline(mid_pt::geometry(Point,4326),ptB::geometry(Point,4326))))  )).geom
                    FROM
                        st_geomfromtext(' %(mid_pt)s ', 4326) mid_pt,
                        st_geomfromtext(' %(street)s ', 4326) street,
                        st_geomfromtext(' %(line_seg_pt)s ', 4326) line_seg_pt,
                        ST_Distance_Spheroid(mid_pt,line_seg_pt,
                            'SPHEROID["WGS 84",6378137,298.257223563]') dist,
                        ST_Project(mid_pt,dist+(dist*1.1),%(ang1)s) ptA,
                        ST_Project(mid_pt,dist+(dist*1.1),%(ang2)s) ptB
                                                """.replace('\n','')%{'mid_pt' : str(lot_seg_mid_pt.geom[0].to_wkt()),
                                                                      'line_seg_pt' : str(street_seg_mid_pt.geom[0].to_wkt()),
                                                                      'street'  : LINE,
                                                                      'ang1' : str(deg_to_rad(90-(360-rad_to_deg(seg_angle)))),
                                                                      'ang2' : str(deg_to_rad(270-(360-rad_to_deg(seg_angle))))},
                                              engine).geom
            #
                if len(this_lot_pt)==0 or len(lot.geom)>1:
                    print 'skipping lot:',lots.bbl.tolist()[i]
                    if len(lot.geom)>1:
                        print '\t\tabove too much for single lot'
                else:

                    if show_some_detail==True: A.append(this_lot_pt[0]) # taking first value...
                    lot_pts.append({'bbl':bbl,
                                    'geom':this_lot_pt[0]})

                    ### Line Connecting Lot to Street
                    line_lot_to_street = gd.read_postgis("""
                        SELECT ST_ShortestLine(lot_pt,mid_pt) geom
                        FROM
                            st_geomfromtext(' %(lot_pt)s ',4326) lot_pt,
                            st_geomfromtext(' %(mid_pt)s ',4326) mid_pt
                        """.replace('\n','')%{'mid_pt'  : str(lot_seg_mid_pt.geom[0].to_wkt()),
                                              'lot_pt'  : this_lot_pt[0]},engine)
            #
                    if len(line_lot_to_street.geom)>1:
                        gd.GeoSeries(line_lot_to_street.geom).plot()
                        print 'too much for Line Connecting Lot to Street'
                        raise SystemError
                    if show_some_detail==True: A.append(line_lot_to_street.geom[0])


            if show_some_detail==True:
                d=gd.GeoSeries(A).plot(fig_size=(26,22),
                                       save_fig_path=save_fig_path,
                                       save_and_show=False)

            if show_some_detail==True: me = raw_input('y?')
            else: me='y'

            if (me=='y' or me=='yq') and lot_pts!=[]:
                c = gd.GeoDataFrame(lot_pts)
                d=[]
                dev=[d.extend(zip(*it.coords.xy)) for it in c.geom]
                D = pd.DataFrame({'bbl':c.bbl.tolist(),'xy':d})
                D['x'],D['y'] = zip(*D.xy)
                D.to_sql('temp',engine,if_exists='append')

                print 'added street_name\t',street_name,'with',len(lot_pts),'lots'
                if me=='yq': raise SystemError
                else:
                    lot_pts,A,skip = [],[],False
                    plt.clf()
            else:
                if me!='y': raise SystemExit


engine.execute("""
UPDATE lot_pts l
SET geom = st_setsrid(st_makepoint(t.x,t.y),4326) 
FROM temp t 
WHERE l.bbl = t.bbl
""")
print 'DONE!'



In [ ]:
### Added index for fuzzystrmatch

    CREATE INDEX idx_places_sndx_loc_name ON address_idx USING btree (soundex(addr));

In [ ]:
### Get all MN zipcodes
py_path.append(abspath(workDir+'../../html'))
import HTML_API as H

from urllib import urlopen
src = 'http://www.newyorkzipcodes.com/city-manhattan/default.aspx'
p = urlopen(src)
x = p.read()
# print x

h = H.getSoup(x)
z = h.findAll('table',attrs={'width':"100%",'border':"0",'cellpadding':"2"})[0]
# print z.prettify()
mn_zipcodes = []
for it in z.findAll('td'):
    t = it.contents[0]
    try:
        if len(t.strip())==5:
            mn_zipcodes.append(int(t))
    except: pass
str(sorted(mn_zipcodes))

In [ ]:
### Check for missing ways in address_idx
T.update( {'ways_tbl' : 'lws',             # %(ways_tbl)s
           'addr_idx' : 'address_idx',     # %(addr_idx)s
           } )

#  'debug': True,
#  'growl_notice': True,
#  'guid': '99fff11',
#  'oldest_comments': '270',
#  'tmp_tbl': 'tmp_99fff11',
#  'today': datetime.datetime(2015, 3, 5, 13, 55, 19, 546317),
#  'transcation_cnt': '100',
#  'user': 'ub2',


chk_for_missing_ways_in_address_idx="""

SELECT count(*) 

from
--    %(ways_tbl)s t,
    %(addr_idx)s t,
    (select array_agg(distinct w.street) all_w_streets from %(ways_tbl)s w) as f1
--    (select array_agg(distinct idx.street) all_idx_streets from %(addr_idx)s idx) as f2

WHERE all_w_streets && array[t.street]
--WHERE all_idx_streets && array[t.street]


""" % T
%sql postgres@routing $chk_for_missing_ways_in_address_idx

In [ ]:
### PROVE ALL STREETS IN WAY TABLE ARE IN ADDRESS_IDX

a=pd.read_sql(""" SELECT count(distinct t.street) cnt
                  FROM 
                      %(ways_tbl)s t,
                      (select array_agg(distinct idx.street) all_idx_streets from %(addr_idx)s idx) as f2
                  WHERE not all_idx_streets && array[t.street]
              """ % T,routing_eng).cnt[0]

addrs_in_way_tbl_but_not_isx_tbl = a
try:
    assert addrs_in_way_tbl_but_not_isx_tbl==0
except AssertionError:
#     add_streets_to_addr_ixs(T['ways_tbl'])
#     process_street_names_and_update_idx()
#     re_run_this_test()
    print 'OK'



## Other Tables

In [ ]:
# Table Creation Libraries

from os                             import environ          as os_environ
from os                             import path             as os_path
from sys                            import path             as py_path
py_path.append(                     os_path.join(os_environ['BD'],'geolocation'))
from f_postgres import Tables
T = Tables()

In [ ]:
### __ create mnv (restaurant inspection results)
m = pd.read_excel('https://data.cityofnewyork.us/api/views/xx67-kt59/rows.xls?accessType=DOWNLOAD')
# was not available 2014.11.19@08:00:00-0500

c="""
DROP TABLE IF EXISTS mnv;

CREATE TABLE mnv (
    building text,
    camis double precision,
    vend_name text,
    clean_street text,
    cuisinecode double precision,
    dba text,
    inspdate timestamp without time zone,
    street text,
    zipcode double precision,
    id integer primary key,
    recog_street boolean,
    recog_addr boolean DEFAULT false,
    bbl integer,
    phone bigint,
    norm_addr text,
    seamless_id bigint,
    yelp_id text,
    address text,
    geom geometry(Point,4326),
    lot_cnt integer DEFAULT 1
  );

UPDATE mnv SET id = nextval(pg_get_serial_sequence('mnv','id'));
"""

In [ ]:
### __ create scrape_lattice (~ about 2 min)

T.make_scrape_lattice(pt_buff_in_miles=0.075,lattice_table_name='yelp_lattice_table')


In [ ]:
### __ create pluto centroids table
conn.set_isolation_level(0)
cur.execute("""
    drop table if exists pluto_centroids;

    create table pluto_centroids (
        pc_gid serial primary key,
        p_gid integer,
        bbl numeric,
        geom geometry(Point,4326)
        );

    UPDATE pluto_centroids 
    SET pc_gid = nextval(pg_get_serial_sequence('pluto_centroids','pc_gid'));
 
    INSERT INTO pluto_centroids (p_gid,geom,bbl)
    SELECT p.gid,st_centroid(p.geom),p.bbl
    FROM pluto p;

                """)

In [ ]:
### __ create 'geocoded'
cmd="""
CREATE TABLE geocoded
(
    id serial primary key,
    addr_valid boolean,
    form_addr text,
    geometry text,
    orig_addr text,
    partial_match boolean,
    res_data text,
    res_i bigint,
    lat double precision,
    lon double precision,
    seamless_id bigint,
    yelp_id bigint,
    mnv_id bigint,
    bbl numeric(10),
    lot_idx numeric(10,5)
);

update geocoded set id = nextval(pg_get_serial_sequence('geocoded','id'));
"""
conn.set_isolation_level(0)
cur.execute(cmd)

In [ ]:
### __ created lot_pts

Ran this in pgAdmin3:

    table lot_pts as 
    select p.Borough,p.Block,p.Lot,p.Address,l.SegmentID,l.Street 
    from pluto p 
    inner join lion_ways l 
    on l.Street ~* p.Address
    
    ALTER TABLE lot_pts ADD COLUMN id SERIAL;
    UPDATE lot_pts SET id = nextval(pg_get_serial_sequence('lot_pts','id'));
    ALTER TABLE lot_pts ADD PRIMARY KEY (id);

    
    create table lot_pts (
        Borough		character varying(2),
        Block 		integer,
        Lot 		smallint,
        bldg_num 	integer,
        bldg_street 	character varying(28),
        Address 	character varying(28),
        SegmentID 	character varying(254),
        Street 		character varying(254),
        CONSTRAINT bbl PRIMARY KEY(Borough,Block,Lot)
    );

In [ ]:
### __ create the topology:
engine.execute("""
    SELECT pgr_createTopology('lion_ways_slim', 0.0005, 'geom', 'gid');
    REINDEX INDEX lion_slim_source_idx;
    REINDEX INDEX lion_slim_target_idx;
    """)

# Custom PostGIS Functions

In [ ]:
### Hausdorff Distance
coords = zip(*lot.boundary[0].coords.xy)
for i in range(1,len(coords)):

    print i,pd.read_sql("""
    SELECT 
        st_HausdorffDistance(
            st_MakeLine(
                st_MakePoint( %(x1y1)s ),
                st_MakePoint( %(x2y2)s )),
            st_geomfromtext(' %(street)s ') ) dist;
    """.replace('\n','').replace('\t','')%{  	'street':str(tmp_part_line.geom[0]),
            'x1y1' : str(coords[i-1]).strip('()'),
            'x2y2' : str(coords[i]).strip('()')},
    engine).dist[0]

In [ ]:
### Azimuth Angles
coords = zip(*lot.boundary[0].coords.xy)
for i in range(1,len(coords)):

	print i,pd.read_sql("""
	SELECT 
    	ST_Azimuth(
            	st_MakePoint( %(x1y1)s ),
            	st_MakePoint( %(x2y2)s )) seg_ang,
        ST_Azimuth(
        	ST_StartPoint( st_geomfromtext(' %(street)s ',4326) ),
            ST_EndPoint( st_geomfromtext(' %(street)s ',4326) )) street_ang
	""".replace('\t','').replace('\n','')%{  	
            'street':str(tmp_part_line.geom[0]),
        	'x1y1' : str(coords[i-1]).strip('()'),
        	'x2y2' : str(coords[i]).strip('()')
    },	engine)

    
plt.clf()
A=[]
#A.append(line_lot_to_street.geom[0])
#A.append(this_lot_pt[0])
A.append(lot_seg_mid_pt.geom[0])
#A.append(lot.geom[0])
#A.append(lot.boundary[0])
A.extend(tmp_part_line.geom)
A.extend(gd.read_postgis("""
	SELECT st_MakeLine(
        st_MakePoint( %(x1y1)s ),
        st_MakePoint( %(x2y2)s )) geom
	""".replace('\n','').replace('\t','')%{
        	'x1y1' : str(coords[7]).strip('()'),
        	'x2y2' : str(coords[2]).strip('()')},
	engine).geom)
gd.GeoSeries(A).plot()



# In[1]:

In [ ]:
### Plot Geoms Using z_get_way_box
A = []
A.extend(gd.read_postgis("select z_get_way_box('14 st','10 ave','59 st','1 ave') geom",routing_eng).geom)
# A.extend(gd.read_postgis("select ST_MinimumBoundingCircle(z_get_way_box('14 st','10 ave','59 st','1 ave')) geom",engine).geom)
# A.extend(
#     gd.read_postgis("""
#     select st_closestpoint(st_intersection(st_buffer(ST_SetSRID(pt,4326)::geometry,.003),nodes) ,pt) geom
#     from
#         (select ST_SetSRID(st_geomfromtext('POINT (-73.97832978915525 40.75142795208318)'),4326)::geometry pt) as t1,
#         (select the_geom nodes from mn_ways_vertices_pgr) as t2
#         """.replace('\n',''),routing_eng).geom)
# A.extend(g.geom)  ## combined numbered streets (e/w)
# A.extend(gd.GeoDataFrame(save_for_later).geom)  ## numbered streets not combined

In [ ]:
### __ MN_VERTICES IN BOX
cmd =   """
        select the_geom geom 
        from 
        
            mn_ways_vertices_pgr,
            
            (select z_get_way_box('14 st','10 ave','59 st','1 ave') box_geom) as t
        where st_within(the_geom,box_geom)
        """.replace('\n',' ')
# A.extend(gd.read_postgis(cmd,routing_eng).geom)

In [ ]:
### __ MN_WAYS IN BOX
cmd =   """
        select st_intersection(m.the_geom,box_geom) geom 
        from 
            (select z_get_way_box('14 st','10 ave','59 st','1 ave') box_geom) as t,
            
            mn_ways m
            
        """.replace('\n',' ')
A.extend(gd.read_postgis(cmd,routing_eng).geom)

In [ ]:
### __ AVENUES IN BOX
cmd =   """
        select st_intersection(geom,box_geom) geom 
        from 
            address_idx,
            (select z_get_way_box('14 st','10 ave','59 st','1 ave') box_geom) as t
        where geom is not null and street ~* 'ave'""".replace('\n',' ')
A.extend(gd.read_postgis(cmd,routing_eng).geom)

In [ ]:
### __ SL VENDORS IN BOX
cmd =   """
        select st_intersection(geom,box_geom) geom 
        from 
            seamless,
            (select z_get_way_box('14 st','10 ave','59 st','1 ave') box_geom) as t
        where geom is not null""".replace('\n',' ')
A.extend(gd.read_postgis(cmd,routing_eng).geom)

# geom_inside_street_box(ways,geom_table,geom_label,table_cols=None,conditions=None)

In [ ]:
### (cont'd,other) Plot Geoms Using z_get_way_box
A=[]
pluto_as_single_poly="""
SELECT ST_Buffer(ST_ConvexHull((ST_Collect(the_geom))), 0.0005) as geom
FROM ( 
    SELECT (ST_Dump( unnest(  all_pluto_geoms ) )).geom the_geom 
    from (select array_agg(p.geom) all_pluto_geoms from pluto p) as fi1
      ) as t
"""
A.extend(gd.read_postgis(pluto_as_single_poly,routing_eng).geom)

all_yelp_scrape="""
SELECT geom from yelp_scrape_lattice;
"""
A.extend(gd.read_postgis(all_yelp_scrape,routing_eng).geom)
%matplotlib inline
gd.GeoSeries(A).plot(alpha=0.1)

# Useful SQL/PG SQL Queries

In [ ]:
Get PostGIS version:

    SELECT PostGIS_full_version();

Get GPS Coordinates:

    SELECT ST_X(geom), ST_Y(geom), ST_AsText(geom) FROM lion_nodes LIMIT 1;

Create Topology for lion_ways:

    ALTER TABLE lion_ways ADD COLUMN "source" integer;
    ALTER TABLE lion_ways ADD COLUMN "target" integer;
    SELECT pgr_createTopology('lion_ways', 0.00001, 'geom', 'gid');
    CREATE INDEX lion_source_idx ON lion_ways("source");
    CREATE INDEX lion_target_idx ON lion_ways("target");

Add Routing Prerequisites:

    ALTER TABLE lion_ways ADD COLUMN reverse_cost double precision;
    UPDATE lion_ways SET reverse_cost = shape_leng;


Get data with street name containing 'madison':

    SELECT * from lion_ways where lion_ways.street ~* 'madison';
    SELECT * from pluto where pluto.address ~* 'madison' limit 1;

Copy from Pandas Import Table to Pluto:

    UPDATE pluto SET bldg_num = 0;
    update pluto set bldg_num = info.bldg_num
        from pluto_info info where info.gid = pluto.gid;

    UPDATE pluto SET bldg_street = '';
    update pluto set bldg_street = info.bldg_street
        from pluto_info info where info.gid = pluto.gid;

# Useful PostGIS Functions:

## Conversion Functions

In [ ]:
    multi-poly to poly:

        select ST_AsText((ST_Dump(pluto.geom)).geom) from pluto where block=860 and lot=26 limit 100;
    
    
    Linear Referencing

        float ST_LineLocatePoint(geometry a_linestring, geometry a_point);

## Geometry Constructors

In [ ]:
Polygon geometry ST_BdPolyFromText(text WKT MultiLineString, integer srid)
    
    MultiPolygon geometry ST_BdMPolyFromText(text WKT MultiLineString, integer srid);
    
    geometry ST_GeomCollFromText(text WKT, integer srid); 
    geometry ST_GeomCollFromText(text WKT);
    
    geometry ST_GeomFromText(text WKT);
    geometry ST_GeomFromText(text WKT, integer srid);
    
    geometry ST_GeomFromGeoJSON(text geomjson);
    
    geometry ST_LineFromMultiPoint(geometry aMultiPoint);
    
    geometry ST_LineFromText(text WKT);
    geometry ST_LineFromText(text WKT, integer srid);
    
        SELECT ST_LineFromText(’LINESTRING(1 2, 3 4)’) AS aline, ST_LineFromText(’POINT(1 2)’) AS null_return;

    box2d ST_MakeBox2D(geometry pointLowLeft, geometry pointUpRight);
    
        SELECT feature_id, feature_name, the_geom
        FROM features
        WHERE the_geom && ST_SetSRID(ST_MakeBox2D(ST_Point(-989502.1875, 528439.5625),
          ST_Point(-987121.375 ,529933.1875)),2163)

    point-or-line geometry ST_MakeLine(geometry set geoms);
    array of point-or-lines geometry ST_MakeLine(geometry geom1, geometry geom2); 
    two point-or-line geometry ST_MakeLine(geometry[] geoms_array);
    
    closed lines geometry ST_MakePolygon(geometry linestring);
    closed lines geometry ST_MakePolygon(geometry outerlinestring, geometry[] interiorlinestrings);
    
    geometry ST_MakePoint(double precision x, double precision y);
    
    geometry ST_MakePointM(float x, float y, float m);
    
    geometry ST_MLineFromText(text WKT, integer srid); 
    geometry ST_MLineFromText(text WKT);

    ST_MPointFromText
    ST_MPolyFromText
    
    geometry ST_Point(float x_lon, float y_lat);
    
    geometry ST_PointFromText(text WKT);
    geometry ST_PointFromText(text WKT, integer srid);
    
    geometry ST_PolygonFromText(text WKT);
    geometry ST_PolygonFromText(text WKT, integer srid);
    
    text GeometryType(geometry geomA);
        #returns geom_type as text
    text ST_GeometryType(geometry g1);
        # returns string with "ST_"
        
    geometry ST_Boundary(geometry geomA);
    
    geometry ST_StartPoint(geometry geomA);
    
    boolean ST_EndPoint(geometry g);
        # returns linestring endpoint
        SELECT ST_AsText(ST_EndPoint(’LINESTRING(1 1, 2 2, 3 3)’::geometry));
    
    geometry ST_Envelope(geometry g1);
        #Returns the float8 minimum bounding box for the supplied geometry, as a geometry. 
        #The polygon is defined by the corner points of the bounding box 
        #((MINX, MINY), (MINX, MAXY), (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)).
    
    linestring geometry ST_ExteriorRing(geometry a_polygon);
        #polygon only
    
    linestring geometry ST_InteriorRingN(geometry a_polygon, integer n);
    
    boolean ST_IsClosed(geometry g);
    
    boolean ST_IsCollection(geometry g);
        • GEOMETRYCOLLECTION
        • MULTI{POINT,POLYGON,LINESTRING,CURVE,SURFACE} 
        • COMPOUNDCURVE
    
    boolean ST_IsEmpty(geometry geomA);
    
    boolean ST_IsRing(geometry g);
        Returns TRUE if this LINESTRING is both 
        ST_IsClosed (ST_StartPoint((g)) ~= ST_Endpoint((g))) and 
        ST_IsSimple (does not self intersect).
    
    boolean ST_IsSimple(geometry geomA);
        #no anomalous geometric points, such as self intersection or self tangency
    
    float ST_M(geometry a_point);
    
    integer ST_NPoints(geometry g1);
        #Return the number of points (vertexes) in a geometry.
    
    integer ST_NumPoints(geometry g1);
        #number of points in an ST_LineString or ST_CircularString value.
    
    geometry ST_PointN(geometry a_linestring, integer n);
        #NULL if there is no linestring in the geometry.
    
    text ST_Summary(geometry g);
    
    float ST_X(geometry a_point);

## Geometry Editors

In [ ]:
    geometry ST_RemovePoint(geometry linestring, integer offset);
    geometry ST_AddPoint(geometry linestring, geometry point);
    geometry ST_AddPoint(geometry linestring, geometry point, integer position);
        Adds a point to a LineString before point <position> (0-based index). 
        Third parameter can be omitted or set to -1 for appending.
        
    [ LineStrings ] geometry ST_LineMerge(geometry amultilinestring);
    
    geometry ST_CollectionHomogenize(geometry collection);
    
    geometry ST_Multi(geometry g1);
        # g1 --> ST_GeomFromText
    
    geometry ST_Reverse(geometry g1);
    
    geometry ST_Segmentize(geometry geom, float max_segment_length);
    
    geometry ST_SetPoint(geometry linestring, integer zerobasedposition, geometry point);
    
    geometry ST_Snap(geometry input, geometry reference, float tolerance);
        # vertices and segments of a geometry with another Geometry’s vertices
    
    geometry ST_TransScale(geometry geomA, float deltaX, float deltaY, float XFactor, float YFactor);
        Translates the geometry using the deltaX and deltaY args, 
        then scales it using the XFactor, YFactor args, working in 2D only.

## Geometry Outputs

In [ ]:
    text ST_AsEWKT(geometry g1);
    
    text ST_AsGeoJSON(geometry geom, integer maxdecimaldigits=15, integer options=0);
    
    text ST_AsText(geometry g1);
    
    text ST_AsLatLonText(geometry pt, text format='');
        # Returns the Degrees, Minutes, Seconds representation of the point.
        # SELECT (ST_AsLatLonText(’POINT (-3.2342342 -2.32498)’, ’D.DDDD degrees C’));

## Operators

In [ ]:
Match "&&" 
    &&( geometry A , geometry B );
    Overlap "&<",">&"
    Overlap/below "&<|"
    Strictly right/left "<<" ">>"
    *Equals "=" ONLY returns true if bounding box is same, geom NOT matter
    Contained by "@"
    Contains another "~"
        ~( geometry A , geometry B );
    Distance "<->"
    Distance b/t bounding boxes "<#>"

## Relationships & Measurements

In [ ]:
    float ST_Area(geometry g1);
    
    geometry ST_Centroid(geometry g1);
    
    geometry ST_ClosestPoint(geometry g1, geometry g2);

    boolean ST_Contains(geometry geomA, geometry geomB);
    
        A circle within a circle
            SELECT ST_Contains(smallc, bigc) As smallcontainsbig,
                 ST_Contains(bigc,smallc) As bigcontainssmall,
                 ST_Contains(bigc, ST_Union(smallc, bigc)) as bigcontainsunion,
                 ST_Equals(bigc, ST_Union(smallc, bigc)) as bigisunion,
                 ST_Covers(bigc, ST_ExteriorRing(bigc)) As bigcoversexterior,
                 ST_Contains(bigc, ST_ExteriorRing(bigc)) As bigcontainsexterior
            FROM (SELECT ST_Buffer(ST_GeomFromText(’POINT(1 2)’), 10) As smallc,
                   ST_Buffer(ST_GeomFromText(’POINT(1 2)’), 20) As bigc) As foo;

    boolean ST_ContainsProperly(geometry geomA, geometry geomB);
        
        a circle within a circle
            SELECT ST_ContainsProperly(smallc, bigc) As smallcontainspropbig, 
                ST_ContainsProperly(bigc,smallc) As bigcontainspropsmall,
                ST_ContainsProperly(bigc, ST_Union(smallc, bigc)) as bigcontainspropunion, 
                ST_Equals(bigc, ST_Union(smallc, bigc)) as bigisunion,
                ST_Covers(bigc, ST_ExteriorRing(bigc)) As bigcoversexterior, 
                ST_ContainsProperly(bigc, ST_ExteriorRing(bigc)) As bigcontainsexterior
            FROM (SELECT ST_Buffer(ST_GeomFromText(’POINT(1 2)’), 10) As smallc, 
                ST_Buffer(ST_GeomFromText(’POINT(1 2)’), 20) As bigc) As foo;

    boolean ST_CoveredBy(geometry geomA, geometry geomB);
    boolean ST_Covers(geometry geomA, geometry geomB);
        Returns 1 (TRUE) if no point in Geometry/Geography B is outside Geometry/Geography A
        
--    boolean ST_Crosses(geometry g1, geometry g2);*

    integer ST_LineCrossingDirection(geometry linestringA, geometry linestringB);
    
    boolean ST_Disjoint( geometry A , geometry B );
    
    float ST_Distance(geometry g1, geometry g2);
        # the 2-D minimum cartesian distance between two geometries in projected spatial ref units
    
    float ST_MaxDistance(geometry g1, geometry g2);
        # SELECT ST_MaxDistance(’POINT(0 0)’::geometry, ’LINESTRING ( 2 0, 0 2 )’::geometry);
    
    boolean ST_DFullyWithin(geometry g1, geometry g2, double precision distance);
        # Returns true if the geometries is fully within the specified distance of one another.
    
    boolean ST_Intersects( geometry geomA , geometry geomB );
        # tolerance is 0.00001 meters (so any points that close are considered to intersect)
    
    float ST_Length(geometry a_2dlinestring);
        # multilinestring too
    
    geometry ST_ShortestLine(geometry g1, geometry g2);
    geometry ST_LongestLine(geometry g1, geometry g2);
    
    boolean ST_Overlaps(geometry A, geometry B);
    
    float ST_Perimeter(geometry g1);
        ST_Surface or ST_MultiSurface geometry or geography. (Polygon, Multipolygon).
        # use ST_Length for linestrings
    
    boolean ST_Touches(geometry g1, geometry g2);
    
    boolean ST_Within(geometry A, geometry B);
        TRUE if geometry A is completely inside geometry B.

    geometry ST_Extrude(geometry geom, float x, float y, float z);

## Geometry Processing

In [ ]:
    geometry ST_Buffer(geometry g1, float radius_of_buffer);
    geometry ST_Buffer(geometry g1, float radius_of_buffer, integer num_seg_quarter_circle); 
    geometry ST_Buffer(geometry g1, float radius_of_buffer, text buffer_style_parameters);
        #buffer_style options: quad_segs=#,endcap=round|flat|square,join=round|mitre|bevel,mitre_limit=#.#
    
    geometry ST_BuildArea(geometry A);
    
    geometry ST_Collect(geometry set g1field); 
    geometry ST_Collect(geometry g1, geometry g2); 
    geometry ST_Collect(geometry[] g1_array);
    
        SELECT stusps,
             ST_Multi(ST_Collect(f.the_geom)) as singlegeom
           FROM (SELECT stusps, (ST_Dump(the_geom)).geom As the_geom
            FROM
            somestatetable ) As f
        GROUP BY stusps
    
    geometry ST_ConvexHull(geometry geomA);
        # minimum convex geometry that encloses all geometries within the set.
    
    geometry ST_Difference(geometry geomA, geometry geomB);
    
    geometry ST_Intersection( geometry geomA , geometry geomB );
    
    geometry ST_MakeValid(geometry input);
    
    geometry ST_MinimumBoundingCircle(geometry geomA, integer num_segs_per_qt_circ=48);
    
    geometry ST_Polygonize(geometry set geomfield); geometry ST_Polygonize(geometry[] geom_array);
    
    geometry ST_RemoveRepeatedPoints(geometry geom);
    
--    geometry ST_SharedPaths(geometry lineal1, geometry lineal2);*

    geometry ST_UnaryUnion(geometry geom);